In [1]:
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:

my_url = 'https://www.caribank.org/our-work/projects-map/list-projects'
#set local path to download
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

#parse HTML
page_soup = soup(page_html, "html.parser")


In [3]:
#find all urls by beautifulsoup
links = [a['href'] for a in page_soup.find_all('a', href=True)]

In [4]:
#find project urls
lst=[]
for i in links:
    if '/our-work/projects-map/' in i:
        lst.append(i)

In [5]:
lst[2:]

['/our-work/projects-map/support-establishment-specialised-micro-and-small-business-financing-assessment-unit-within',
 '/our-work/projects-map/essex-valley-irrigation-project',
 '/our-work/projects-map/strengthening-food-safety-management-systems',
 '/our-work/projects-map/energy-efficiency-measures-and-solar-photovoltaic-plant',
 '/our-work/projects-map/road-and-bridge-rehabilitation-loubiere-bagatelle-road-and-road-network-and-road-safety-assessment',
 '/our-work/projects-map/road-infrastructure-rehabilitation',
 '/our-work/projects-map/education-quality-improvement-project',
 '/our-work/projects-map/water-supply-expansion-and-sewerage-improvement-project',
 '/our-work/projects-map/skills-development-and-employability-project',
 '/our-work/projects-map/technical-assistance-geomatics-capacity-enhancement-disaster-risk-management',
 '/our-work/projects-map/street-and-flood-light-retrofitting-project',
 '/our-work/projects-map/power-project-electricity-system-upgrade-and-expansion',
 '

In [5]:
#Revise all urls so we can access directly
final_link=list(map(lambda x:'https://www.caribank.org'+x,lst[2:]))

In [6]:
final_link[:5]

['https://www.caribank.org/our-work/projects-map/support-establishment-specialised-micro-and-small-business-financing-assessment-unit-within',
 'https://www.caribank.org/our-work/projects-map/essex-valley-irrigation-project',
 'https://www.caribank.org/our-work/projects-map/strengthening-food-safety-management-systems',
 'https://www.caribank.org/our-work/projects-map/energy-efficiency-measures-and-solar-photovoltaic-plant',
 'https://www.caribank.org/our-work/projects-map/road-and-bridge-rehabilitation-loubiere-bagatelle-road-and-road-network-and-road-safety-assessment']

In [7]:
#Test: Try to put one project data into a table
link='https://www.caribank.org/our-work/projects-map/support-establishment-specialised-micro-and-small-business-financing-assessment-unit-within'
bluClient = uReq(link)
blupage_html = bluClient.read()
bluClient.close()
blupage_soup = soup(blupage_html, 'html')

In [8]:
#find overview 
a=blupage_soup.find_all('div',{'class':['field--wrap field--node field--node--full field--node--body']})
b=str(a).replace('<p>','</p>')
description=b.split('</p>')[1]


In [9]:
#find date
#blupage_soup.find_all('div',{'class':['field--label']})
c=blupage_soup.find_all('div',{'class':['field--wrap field--node--field-date-of-approval field-display--above']})
d=str(c).replace('<div>','</div>')
date=d.split('</div>')[2]


In [10]:
# find country
#need try_catch 
e=blupage_soup.find_all('div',{'class':['node--contry-tag']})
f=str(e).split('hreflang="en">')[1]
country=f.split('</a>')[0]


In [11]:
#find budget
g=blupage_soup.find_all('div',{'class':['field--wrap field--node--field-cdb-approved-budget field-display--above']})
h=str(g).replace('<div>','</div>')
approved_total=h.split('</div>')[2]

In [12]:
# find status 
gg=blupage_soup.find_all('div',{'class':['field--wrapp']})
status=str(gg).split('</span>')[1].split('</div>')[0]


In [13]:
table=pd.DataFrame([[description,date,country,status]],columns=['description','date','country','status'])
table

,description,date,country,status
0,This Project is expected to contribute to incr...,"May, 2016",Haiti,Active


In [14]:
#Put all projects into a table
big_table=pd.DataFrame()
for i in final_link:
    bluClient = uReq(i)
    blupage_html = bluClient.read()
    bluClient.close()
    blupage_soup = soup(blupage_html, 'html')
    #get name
    name=i.split('/')[-1]
    #find overview 
    a=blupage_soup.find_all('div',{'class':['field--wrap field--node field--node--full field--node--body']})
    b=str(a).replace('<p>','</p>')
    description=b.split('</p>')[1]
    #find date
    c=blupage_soup.find_all('div',{'class':['field--wrap field--node--field-date-of-approval field-display--above']})
    d=str(c).replace('<div>','</div>')
    date=d.split('</div>')[2]
    # find country
    #need try_catch 
    e=blupage_soup.find_all('div',{'class':['node--contry-tag']})
    f=str(e).split('hreflang="en">')[1]
    country=f.split('</a>')[0]
    #find budget
    g=blupage_soup.find_all('div',{'class':['field--wrap field--node--field-cdb-approved-budget field-display--above']})
    h=str(g).replace('<div>','</div>')
    approved_total=h.split('</div>')[2]
    # find status 
    gg=blupage_soup.find_all('div',{'class':['field--wrapp']})
    status=str(gg).split('</span>')[1].split('</div>')[0]
    table=pd.DataFrame([[name,i,description,date,country,status,approved_total]],columns=['name','url','description','date','country','status','approved_total'])
    big_table=pd.concat([big_table,table],axis=0)
    print('{}finished iteration '.format(name))

support-establishment-specialised-micro-and-small-business-financing-assessment-unit-withinfinished iteration 
essex-valley-irrigation-projectfinished iteration 
strengthening-food-safety-management-systemsfinished iteration 
energy-efficiency-measures-and-solar-photovoltaic-plantfinished iteration 
road-and-bridge-rehabilitation-loubiere-bagatelle-road-and-road-network-and-road-safety-assessmentfinished iteration 
road-infrastructure-rehabilitationfinished iteration 
education-quality-improvement-projectfinished iteration 
water-supply-expansion-and-sewerage-improvement-projectfinished iteration 
skills-development-and-employability-projectfinished iteration 
technical-assistance-geomatics-capacity-enhancement-disaster-risk-managementfinished iteration 
street-and-flood-light-retrofitting-projectfinished iteration 
power-project-electricity-system-upgrade-and-expansionfinished iteration 
second-road-infrastructure-rehabilitation-projectfinished iteration 
street-light-retrofitting-pro

In [15]:
big_table.index=range(len(big_table))

In [16]:
big_table.to_csv('Carribean')